In [75]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
import pandas as pd
import re
from tqdm import tqdm
nest_asyncio.apply()

In [76]:
urls = pd.read_csv("../data/KP_naver_monthlyTop4000_TB.csv")

In [77]:
urls

,Unnamed: 0,0,1,2
0,0,https://n.news.naver.com/mnews/article/021/000...,카드업계도…‘모바일 페이’ 수수료 전쟁 시작된다,[\n\n\n\n\n\n간편결제 시장 日평균 2500억7개 카드사 결제대행업체 설립...
1,1,https://n.news.naver.com/mnews/article/015/000...,카카오페이도 소상공인에 '활짝'…결제 API 공개,"[\n쉽게 가져다 쓰는 오픈 플랫폼 전략""온라인 결제시스템 도입 부담 덜 것""\n\..."
2,2,https://n.news.naver.com/mnews/article/421/000...,"대한항공, 모바일 항공권 결제에 삼성페이 도입",[\n\n\n\n\n대한항공이 도입한 온라인 결제시스템인 카카오페이와 페이코 서비스...
3,3,https://n.news.naver.com/mnews/article/001/000...,카카오페이로 세금납부 가능해진다…모바일 세금납부 확대,"[\n\n\n\n\n [연합뉴스TV제공]국세청, 전국 세무관서장 회의…종합소득세 전..."
4,4,https://n.news.naver.com/mnews/article/001/000...,"카카오페이, 외부 업체도 자유롭게 쓰세요",[\n\n\n\n\n카카오페이 타이틀 화면 &lt;&lt;카카오 제공&gt;&gt;...
...,...,...,...,...
44075,44075,https://n.news.naver.com/mnews/article/003/001...,"횡성군, 연말연시 '희망 2023 나눔캠페인' 추진","[\n기사내용 요약'함께하는 나눔, 지속가능한 강원'\n\n\n\n[횡성=뉴시스]이..."
44076,44076,https://n.news.naver.com/mnews/article/421/000...,청양군 “2022년 2기분 자동차세 내달 2일까지 납부하세요”,[\n은행 현금자동출납기서 고지서없이 납부 가능\n\n\n\n청양군 청사 전경.(청...
44077,44077,https://n.news.naver.com/mnews/article/421/000...,반갑다 코스피 2500…금리압력 약화로 석달만에 회복개장시황,[\n지난 8월 이후 100여일만…네이버-카카오 등 성장주 강세 환율도 4개월여만에...
44078,44078,https://n.news.naver.com/mnews/article/119/000...,고객 찾아가는 증권사...방문판매 서비스 본격화,[\n증권사 PB가 보험사처럼 펀드상품 등 직접 판매“오래전 시스템 구축...파급력...


In [78]:
test = urls.iloc[:, 1:3]
test

,0,1
0,https://n.news.naver.com/mnews/article/021/000...,카드업계도…‘모바일 페이’ 수수료 전쟁 시작된다
1,https://n.news.naver.com/mnews/article/015/000...,카카오페이도 소상공인에 '활짝'…결제 API 공개
2,https://n.news.naver.com/mnews/article/421/000...,"대한항공, 모바일 항공권 결제에 삼성페이 도입"
3,https://n.news.naver.com/mnews/article/001/000...,카카오페이로 세금납부 가능해진다…모바일 세금납부 확대
4,https://n.news.naver.com/mnews/article/001/000...,"카카오페이, 외부 업체도 자유롭게 쓰세요"
...,...,...
44075,https://n.news.naver.com/mnews/article/003/001...,"횡성군, 연말연시 '희망 2023 나눔캠페인' 추진"
44076,https://n.news.naver.com/mnews/article/421/000...,청양군 “2022년 2기분 자동차세 내달 2일까지 납부하세요”
44077,https://n.news.naver.com/mnews/article/421/000...,반갑다 코스피 2500…금리압력 약화로 석달만에 회복개장시황
44078,https://n.news.naver.com/mnews/article/119/000...,고객 찾아가는 증권사...방문판매 서비스 본격화


In [79]:
# 함수 정의
def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
        'referer': refer_url
    }
    
    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'url' : refer_url,
            'text': x['contents'],
            'reply_count' : x['replyCount'], 
            'uid': x['idNo'],
            'uname' : x['userName'],
            'like': x['sympathyCount'], 
            'dislike': x['antipathyCount'],
            'c_time' : x['modTime'],
            'cid': x['commentNo'],
            'pid' : x['parentCommentNo'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments



In [80]:
def get_data(oid, aid) :
    try :
        # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
        # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
        refer_url = f'https://n.news.naver.com/mnews/article/comment/{oid}/{aid}?sid=100'
        comment_url = f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20220623185938&_callback=jQuery33103200637113167857_1656171100524&lang=ko&country=KR&objectId=news{oid}%2C{aid}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&followSize=100&sort=new&includeAllStatus=true&_=1656171100525'

        comments = get_comments(refer_url, comment_url)
        comments = pd.DataFrame(comments)
        # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
        return comments
    except :
        return None


In [81]:
df = pd.DataFrame()
for url, title in tqdm(zip(test['0'], test['1'])):
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
        sid = url[-3:]
        temp = get_data(oid, aid)
        temp['sid'] = sid
        temp['title'] = title
        df =pd.concat([df, temp])
    except:
        print(url)

177it [00:41,  7.30it/s]

https://sports.news.naver.com/news.nhn?oid=442&aid=0000052551


392it [01:21,  7.22it/s]

https://sports.news.naver.com/news.nhn?oid=442&aid=0000054347


660it [02:15,  5.07it/s]

...

42639it [3:32:30,  2.27it/s]

https://n.news.naver.com/mnews/article/008/0004821203?sid=101


43882it [3:40:04,  2.84it/s]

https://sports.news.naver.com/news.nhn?oid=442&aid=0000157051


44080it [3:41:15,  3.32it/s]


In [82]:
df.reset_index(drop = True, inplace = True)
df

,sid,title,url,text,reply_count,uid,uname,like,dislike,c_time,cid,pid
0,105,"카카오, 간편결제 플랫폼 키운다",https://n.news.naver.com/mnews/article/comment...,네이버 페이보다 수수료 낮을까?,0.0,aSij,saep****,1.0,0.0,2017-01-19T17:38:29+0900,808915624,808915624
1,105,"카카오, 간편결제 플랫폼 키운다",https://n.news.naver.com/mnews/article/comment...,공부해야할 과목이 또 하나 늘었군.. - IT종사자 -,0.0,3RpCa,illh****,1.0,0.0,2017-01-19T17:28:18+0900,808906894,808906894
2,105,"분석+ 카카오 vs 네이버, 같은 페이 다른 느낌",https://n.news.naver.com/mnews/article/comment...,"네이버도 정기결제, 포인트 적립 다 똑같이 되고 먼저 됐는데요.",0.0,7YdEe,bity****,1.0,0.0,2017-01-25T21:54:45+0900,814628004,814628004
3,105,"분석+ 카카오 vs 네이버, 같은 페이 다른 느낌",https://n.news.naver.com/mnews/article/comment...,"네이버페이 좋아요.네이버페이체크카드로 교통카드 사용하면 포인트주고, 천원써도 포인...",0.0,PzCe,kshk****,7.0,2.0,2017-01-22T21:19:51+0900,812157994,812157994
4,105,"분석+ 카카오 vs 네이버, 같은 페이 다른 느낌",https://n.news.naver.com/mnews/article/comment...,네이버페이는 편할거라 생각해서 썼는데 포인트도 제한 교통적립 되긴되는건가?카카오페이...,1.0,47W2k,rhks****,4.0,1.0,2017-01-22T17:03:03+0900,812007474,812007474
...,...,...,...,...,...,...,...,...,...,...,...,...
340592,105,"딱히 하락 이유 없는데...'게임 시총 1위' 내려놓은 크래프톤, 매력도↑",https://n.news.naver.com/mnews/article/comment...,신작 바람 하나 없이 '납세 대상' 이런 지출 기사나 나오는데 당연 하락하지 올라가...,0.0,dEylt,evry****,0.0,0.0,2022-12-16T11:53:02+0900,777898445158481946,777898445158481946
340593,105,"딱히 하락 이유 없는데...'게임 시총 1위' 내려놓은 크래프톤, 매력도↑",https://n.news.naver.com/mnews/article/comment...,신작마다 똥갬을 만드니\n 하락하지 아직도 고평가다\n 4만이 적정주가다\n 기래기...,0.0,5hitw,css6****,7.0,1.0,2022-12-16T10:20:14+0900,777892465809031285,777892465809031285
340594,105,"딱히 하락 이유 없는데...'게임 시총 1위' 내려놓은 크래프톤, 매력도↑",https://n.news.naver.com/mnews/article/comment...,욕밖에 안나오는 쓰레기톤,0.0,2BMm0,vy18****,3.0,0.0,2022-12-16T10:05:01+0900,777891485583409159,777891485583409159
340595,105,"딱히 하락 이유 없는데...'게임 시총 1위' 내려놓은 크래프톤, 매력도↑",https://n.news.naver.com/mnews/article/comment...,하락이유가 없긴 왜없어? 배틀그라운드만 성공해놓고 상장하니까 그렇지,0.0,3gEet,stud****,0.0,0.0,2022-12-16T08:14:41+0900,777884377328648407,777884377328648407


In [ ]:
df.to_pickle('../data/KP_naver_comments_monthlyTop4000.pkl')
df.to_csv('../data/KP_naver_comments_monthlyTop4000.csv')